# MSP Italy - EMODnet Exporter

In [6]:
from PIL import Image, ImageDraw, ImageFont
import fiona
import pandas as pd
import geopandas as gpd
import numpy as np

In [7]:
source = "data/planning.gpkg"
for layername in fiona.listlayers(source):
    with fiona.open(source, layer=layername) as src:
        print(layername, len(src))

eu_contries 37
uses_pu 4169
regioni_istat 20
land_EEA_2018 27
areas 3
subareas 27
planning_units 327
pu_uses 14
pu_type 4


In [8]:
msp_areas = gpd.read_file(source, layer="areas")
msp_subareas = gpd.read_file(source, layer="subareas")
planning_units = gpd.read_file(source, layer="planning_units")
pu_uses = gpd.read_file(source, layer="pu_uses").set_index("code")
uses_pu = gpd.read_file(source, layer="uses_pu")
main_uses_pu = uses_pu.query("type=='main'")
pu_uses = gpd.read_file(source, layer="pu_uses").set_index("code")



In [9]:
# extend uses to emodnet names
emodnet_uses = pd.read_csv('data/msp_emodnet_uses.csv')
emodnet_uses['code'] = emodnet_uses["code"].astype(str).str.zfill(4)
emodnet_uses.set_index("code", inplace=True)
df_uses = pd.DataFrame(pu_uses)
pu_uses_ext = pd.concat([df_uses, emodnet_uses], axis=1)
pu_uses_ext

,use_eng,use_ita,label,up_types,sld_style,color,use_ita_short,geometry,emodnet,emo_desc,hilucs,hilucsmsp,complete
0100,Fishery,Pesca,p,100,zoning_elements_pesca.sld,#F4AAAE,Pesca,None,Fishing areas,Planning units primary addressed to fishery,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,yes
1500,Aquaculture,Acquacoltura,a,100,zoning_elements_acquacoltura.sld,#B9829F,Acquacoltura,None,Aquaculture,Planning units primary addressed to aquaculture,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,yes
0200,Marine Traffic and Ports,Trasporto marittimo e portualità,tm,100,zoning_elements_trasporto.sld,#6E7C97,Trasporto mar. portualità,None,Maritime Traffic flows,Planning units,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,NaN
0300,Environmental protecion and resources,Protezione ambiente e risorse naturali,n,100,zoning_elements_protezione.sld,#88BC91,Protez. ambiente e ris.naturali,None,Nature Protection Conservation,Nature Protection Conservation,NaN,http://www.geoportal.ulpgc.es/registro/planned...,no
0400,Energy,Energia,e,100,zoning_elements_energia.sld,#C04B4B,Energia,None,NaN,Energy related planning units,NaN,NaN,NaN
0500,Sand extraction,Prelievo di sabbie,sa,100,zoning_elements_sabbie.sld,#FCD154,Sabbie,None,Raw material extraction,NaN,http://inspire.ec.europa.eu/codelist/HILUCSVal...,http://www.geoportal.ulpgc.es/registro/planned...,no
0600,Military,Difesa,d,100,zoning_elements_difesa.sld,#CCAE99,Difesa,None,Military areas,Military training areas,NaN,http://www.geoportal.ulpgc.es/registro/planned...,yes
0700,Tourism,Turismo costiero e marittimo,t,100,zoning_elements_turismo.sld,#F4A166,Turismo,None,Tourism and recreation,Tourism and recreation,NaN,http://www.geoportal.ulpgc.es/registro/planned...,no
1000,Landscape/Seascape Cultural Heritage,Paesaggio e Patrimonio Culturale,ppc,100,zoning_elements_paesaggio.sld,#778775,Paesaggio e Patrimonio Cult.,None,Cultural heritage,"Landscape and cultural heritage (visibile, und...",NaN,http://www.geoportal.ulpgc.es/registro/planned...,no
1400,Scientific research & Innovation,Ricerca scientifica e innovazione,r,100,zoning_elements_ricerca.sld,#999999,Ricerca sc. e innovaz.,None,Scientific research,Scientific research,NaN,http://www.geoportal.ulpgc.es/registro/planned...,no


In [10]:
# load attribute dictionary for  zoning elements

fulldict = pd.read_csv('data/msp_emodnet_dictionary.csv')
ze_dict = fulldict.query('emodnet_layername == "MSP Zoning Element"')

In [12]:
def get_use(code, key):
    if code=='0000':
        return 'generic'
    r = pu_uses_ext.at[code,key] 
    if type(r)==str:       
        return r
    else:
        return '-'

#TODO single function that get first non null value from dictionary
def get_field_value(fieldname):
    v = ze_dict.query('emodnet_ha_fieldname == "'+ fieldname +'"')['italian_msp_fieldame']
    if v.item().startswith('['):
        l = v.item().strip('[]').split(',')
        s = [x.strip() for x in l]
        return s
    else:
        return v.item()
def get_static_value(fieldname):
    v = ze_dict.query('emodnet_ha_fieldname == "'+ fieldname +'"')['italian_msp_staticvalue']
    return v.item()
key = get_field_value('HilucLU')[1].strip()
r = get_use('0410', key)
print(r)

https://inspire.ec.europa.eu/codelist/HILUCSValue/2_4_4_RenewableEnergyProduction


In [13]:
# function to set different energy use for each PU
energy_uses = {
    'presenza piattaforme':'0410',
    'potenziale rinnovabili': '0420'
}


def get_energy_use(up_code):
    if type(up_code)!= str:
        up_code = up_code.item()
    energy= main_uses_pu.query("use_code_s=='0400'")
    #print(energy)
    curruse = energy.query("pu_code=='" +up_code +"'")
    #print(curruse['notes'])
    usenote = curruse['notes'].item().replace(';',',')
    notelist = usenote.split(',')
    notelist = [x.strip() for x in notelist]
    enerlist =[]
    for n in notelist:
        for e in energy_uses:
            if n == e:
                enerlist.append(energy_uses[e])
    return enerlist



In [32]:
# explode dataframe for primary uses
planning_units['main_uses_list'] = planning_units.main_uses.str.split(',')
# explode dataframe for multiple uses  on the same code, apply only on energy (0400)
planning_units['main_uses_list'] = planning_units.apply(lambda x: get_energy_use(x['Code']) if '0400' in x['main_uses_list'] else x['main_uses_list'], axis=1)
ze = planning_units.explode('main_uses_list')
ze.rename(columns ={'main_uses_list':'usecode'}, inplace=True)
# add emodnet columns
ze['MS'] = 'IT'
ze['LocalID'] = 'IT_' + ze['Code'] + ze['Type']
ze['VersionID'] = '0.1'
ze['OffSource'] = get_static_value('OffSource')
l = get_field_value('HilucLU')
ze['HilucLU'] = ze['usecode'].apply(lambda x: get_use(x,l[1]))
l = get_field_value('hilucsMSP')
ze['hilucsMSP'] = ze['usecode'].apply(lambda x: get_use(x,l[1]))
l = get_field_value('SeaUseName')
ze['SeaUseName'] = ze['usecode'].apply(lambda x: get_use(x,l[1]))
# get sea use function bsed on PU type
seaFunction = {100: 'Allowed', 200:'Priority', 300: 'Priority', 400: 'Reserved'}
ze['SeaUseFct'] = ze['pu_type'].apply(lambda x : seaFunction[x])
l = get_field_value('SeaUseDsc')
ze['SeaUseDsc'] = 
ze['OriginalUseName']
ze['OriginalUseNameEn']
#select only columns relevant for EMODnet data model
sel_cols = [ 'LocalID', 'VersionID','OffSource','MS','HilucLU', 'hilucsMSP','SeaUseName','SeaUseFct','SeaUseDsc', 'geometry']
ze = ze[sel_cols]
ze.head()


,MS,LocalID,VersionID,OffSource,HilucLU,hilucsMSP,SeaUseFct,geometry
0,IT,IT_A/3_01P(t),0.1,https://www.sid.mit.gov.it,-,http://www.geoportal.ulpgc.es/registro/planned...,Priority,"MULTIPOLYGON (((7023412.570 4941571.750, 70201..."
1,IT,IT_A/3_02P(a),0.1,https://www.sid.mit.gov.it,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,Priority,"MULTIPOLYGON (((7025074.508 4954529.347, 70251..."
2,IT,"IT_A/3_04P(n,d)",0.1,https://www.sid.mit.gov.it,-,http://www.geoportal.ulpgc.es/registro/planned...,Priority,"MULTIPOLYGON (((7052426.141 4944327.128, 70523..."
2,IT,"IT_A/3_04P(n,d)",0.1,https://www.sid.mit.gov.it,-,http://www.geoportal.ulpgc.es/registro/planned...,Priority,"MULTIPOLYGON (((7052426.141 4944327.128, 70523..."
3,IT,IT_A/3_05P(t),0.1,https://www.sid.mit.gov.it,-,http://www.geoportal.ulpgc.es/registro/planned...,Priority,"MULTIPOLYGON (((7060232.056 4863396.003, 70602..."


In [29]:
ze.query("LocalID=='IT_A/3_07P(e)'")

,MS,LocalID,VersionID,OffSource,HilucLU,hilucsMSP,SeaUseFct
25,IT,IT_A/3_07P(e),0.1,https://www.sid.mit.gov.it,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,Priority
25,IT,IT_A/3_07P(e),0.1,https://www.sid.mit.gov.it,https://inspire.ec.europa.eu/codelist/HILUCSVa...,http://www.geoportal.ulpgc.es/registro/planned...,Priority
